In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from sqlalchemy import create_engine
import psycopg2

#Store Part I results into DataFrame
#Load the data to a DataFrame
accident_data = pd.read_csv("/Users/erikaberry/Desktop/Traffic_Accidents/data/USTrafficAccidents.csv", encoding="utf-8")
accident_data.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201,2,2/8/16 6:07,2/8/16 6:37,39.928059,-82.831184,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201,2,2/8/16 6:49,2/8/16 7:19,39.063148,-84.032608,NaN,NaN,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201,2,2/8/16 7:39,2/8/16 8:09,39.627781,-84.188354,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day


In [14]:
# Data cleanup

# Filters out dates containing 2016, 2017, & 2019 -- too much data in dataset to handle with laptops
accident_data_revised = accident_data[~accident_data["Start_Time"].str.contains("2016")]
accident_data_revised = accident_data_revised[~accident_data_revised["Start_Time"].str.contains("2017")]
accident_data_revised = accident_data_revised[~accident_data_revised["Start_Time"].str.contains("2018")]

# Splits the Start_Time column into individual columns
accident_data_revised[["Date", "Time"]] = accident_data_revised["Start_Time"].str.split(expand=True)

# Renames columns to be more readable
accident_data_revised = accident_data_revised.rename(columns={"Start_Lat":"Lat", "Start_Lng":"Lng", "Weather_Condition":"Weather"})

# Filters and rearranges dataset to display most useful columns
accident_data_revised = accident_data_revised[["Date", "Time", "Lat", "Lng", "City", "State",
         "County", "Weather", "Nautical_Twilight"]]

# Iterates through all column data to find NaN values and adds them to a list
accident_data_revised.columns[accident_data_revised.isna().any()].tolist()

accident_data_revised = accident_data_revised.reset_index()

accident_data_revised.head(20)

,index,Date,Time,Lat,Lng,City,State,County,Weather,Nautical_Twilight
0,0,2/8/16,5:46,39.865147,-84.058723,Dayton,OH,Montgomery,Light Rain,Night
1,1,2/8/16,6:07,39.928059,-82.831184,Reynoldsburg,OH,Franklin,Light Rain,Night
2,2,2/8/16,6:49,39.063148,-84.032608,Williamsburg,OH,Clermont,Overcast,Day
3,3,2/8/16,7:23,39.747753,-84.205582,Dayton,OH,Montgomery,Mostly Cloudy,Day
4,4,2/8/16,7:39,39.627781,-84.188354,Dayton,OH,Montgomery,Mostly Cloudy,Day
5,5,2/8/16,7:44,40.100590,-82.925194,Westerville,OH,Franklin,Light Rain,Day
6,6,2/8/16,7:59,39.758274,-84.230507,Dayton,OH,Montgomery,Overcast,Day
7,7,2/8/16,7:59,39.770382,-84.194901,Dayton,OH,Montgomery,Overcast,Day
8,8,2/8/16,8:00,39.778061,-84.172005,Dayton,OH,Montgomery,Mostly Cloudy,Day
9,9,2/8/16,8:10,40.100590,-82.925194,Westerville,OH,Franklin,Light Rain,Day


In [15]:
# filter accident_data_sorted to include only NC data

NC_accident_data = accident_data_revised.query('State=="NC"')
NC_accident_data.head()

,index,Date,Time,Lat,Lng,City,State,County,Weather,Nautical_Twilight
512339,512339,12/25/19,4:55,35.599384,-80.531868,Salisbury,NC,Rowan,Fair,Night
512340,512340,12/25/19,6:40,35.771000,-81.329910,Hickory,NC,Catawba,Fair,Day
512341,512341,12/25/19,8:36,35.706032,-81.424911,Hildebran,NC,Burke,Fair,Day
512342,512342,12/25/19,9:49,35.065140,-80.845810,Charlotte,NC,Mecklenburg,Fair,Day
512343,512343,12/25/19,12:10,35.223850,-80.745033,Charlotte,NC,Mecklenburg,Fair,Day


In [16]:
# filter accident_data_sorted to include only Wake County data

Wake_accident_data = NC_accident_data.query('County=="Wake"') 
Wake_accident_data.head()

,index,Date,Time,Lat,Lng,City,State,County,Weather,Nautical_Twilight
512361,512361,12/25/19,12:49,35.799881,-78.727257,Raleigh,NC,Wake,Mostly Cloudy,Day
512363,512363,12/25/19,13:22,35.753876,-78.635849,Raleigh,NC,Wake,Mostly Cloudy,Day
512364,512364,12/25/19,14:16,35.800491,-78.666481,Raleigh,NC,Wake,Mostly Cloudy,Day
512365,512365,12/25/19,15:06,35.838581,-78.324638,Zebulon,NC,Wake,Fair,Day
512367,512367,12/25/19,16:11,35.756947,-78.700005,Raleigh,NC,Wake,Mostly Cloudy,Day


In [17]:
# round off the columns in this manner 
Wake_accidents = Wake_accident_data.round({"Lat":6, "Lng":6})
Wake_accidents.head()

,index,Date,Time,Lat,Lng,City,State,County,Weather,Nautical_Twilight
512361,512361,12/25/19,12:49,35.799881,-78.727257,Raleigh,NC,Wake,Mostly Cloudy,Day
512363,512363,12/25/19,13:22,35.753876,-78.635849,Raleigh,NC,Wake,Mostly Cloudy,Day
512364,512364,12/25/19,14:16,35.800491,-78.666481,Raleigh,NC,Wake,Mostly Cloudy,Day
512365,512365,12/25/19,15:06,35.838581,-78.324638,Zebulon,NC,Wake,Fair,Day
512367,512367,12/25/19,16:11,35.756947,-78.700005,Raleigh,NC,Wake,Mostly Cloudy,Day


In [19]:
Wake_accidents.drop(['index'], axis=1)

,Date,Time,Lat,Lng,City,State,County,Weather,Nautical_Twilight
512361,12/25/19,12:49,35.799881,-78.727257,Raleigh,NC,Wake,Mostly Cloudy,Day
512363,12/25/19,13:22,35.753876,-78.635849,Raleigh,NC,Wake,Mostly Cloudy,Day
512364,12/25/19,14:16,35.800491,-78.666481,Raleigh,NC,Wake,Mostly Cloudy,Day
512365,12/25/19,15:06,35.838581,-78.324638,Zebulon,NC,Wake,Fair,Day
512367,12/25/19,16:11,35.756947,-78.700005,Raleigh,NC,Wake,Mostly Cloudy,Day
...,...,...,...,...,...,...,...,...,...
1047978,3/12/19,19:14,35.838970,-78.673721,Raleigh,NC,Wake,Mostly Cloudy,Day
1047979,3/12/19,19:18,35.677074,-78.624901,Garner,NC,Wake,Mostly Cloudy,Day
1047980,3/12/19,19:25,35.861450,-78.638069,Raleigh,NC,Wake,Scattered Clouds,Day
1047983,3/12/19,19:53,35.837669,-78.673141,Raleigh,NC,Wake,Scattered Clouds,Day


In [25]:
# Export file as a CSV, without the Pandas index, but with the header
Wake_accident_data.to_csv("Wake_Traffic.csv", sep='|', index=False, header=True)

In [9]:
connection_string = "postgres:postgres@localhost:5432/TrafficAccidents"
engine = create_engine(f'postgresql://{connection_string}')

In [10]:
engine.table_names()

['cary_accidents', 'wake_accidents', 'premise']

In [12]:
#pd.read_sql_query('select * from wake_accidents', con=engine).head()